# Getting Started with Data Ingestion and Preparation

Learn how to quickly start using the Iguazio Data Science Platform to collect, ingest, and explore data.

- [Overview](#gs-overview)
  - [Platform Data Containers](#platform-data-containers)
- [Collecting and Ingesting Data](#gs-data-collection-and-ingestion)
  - [Ingesting Data From an External Database to a NoSQL Table Using V3IO Frames](#ingest-from-external-db-to-no-sql-using-frames)
  - [Ingesting Files from Amazon S3](#ingest-from-amazon-s3)
  - [Streaming Data From an External Streaming Engine Using Nuclio](#streaming-data-from-an-external-streaming-engine-using-nuclio)
- [Exploring and Processing Data](#gs-data-exploration-and-processing)
  - [Exploring Data Using Spark DataFrames](#data-exploration-spark)
  - [Exploring Data Using V3IO Frames and pandas DataFrames](#data-exploration-v3io-frames-n-pandas)
  - [Exploring Data Using SQL](#data-exploration-sql)
- [Data Collection and Exploration Getting-Started Example](#getting-started-example)
  - [Step 1: Ingest a Sample CSV File from Amazon S3](#getting-started-example-step-ingest-csv)
  - [Step 2: Convert the Sample CSV File to a NoSQL Table](#getting-started-example-step-convert-csv-to-nosql-table)
  - [Step 3: Run Interactive SQL Queries](#getting-started-example-step-run-sql-queries)
  - [Step 4: Convert the Data to a Parquet Table](#getting-started-example-step-convert-data-to-parquet)
  - [Step 5: Browse the Example Container Directory](#getting-started-example-step-browse-the-examples-dir)
- [Cleanup](#cleanup)
  - [Delete Data](#delete-data)
  - [Release Spark Resources](#release-spark-resources)

<a id="gs-overview"></a>
## Overview

This tutorial explains and demonstrates how to collect, ingest, and explore data with the Iguazio Data Science Platform (**"the platform"**).<br>
For an in-depth overview of the platform and how it can be used to implement a full data science workflow, see the [**platform-overview.ipynb**](../platform-overview.ipynb) tutorial notebook.<br>
For full end-to-end platform use-case application demos, see [**demos**](../demos/README.ipynb) tutorial notebooks directory.

<a id="platform-data-containers"></a>
### Platform Data Containers

Data is stored within data containers in the platform's distributed file system (DFS).
All platform clusters have two predefined containers:

- <a id="default-container"></a> The default **"bigdata"** container.
- <a id="users-container"></a>The **"users"** container, which is designed to contain **&lt;username&gt;** directories that provide individual development environments for storing user-specific data.
  The platform's Jupyter Notebook, Zeppelin, and web-based shell "command-line services" automatically create such a directory for the running user of the service and set it as the home directory of the service environment.
  You can leverage the following environment variables, which are predefined in the platform's command-line services, to access this running-user directory from your code:

  - `V3IO_USERNAME` &mdash; set to the username of the running user of the Jupyter Notebook service.
  - `V3IO_HOME` &mdash; set to the running-user directory in the "users" container &mdash; **users/&lt;running user&gt;**.
  - `V3IO_HOME_URL` &mdash; set to the fully qualified `v3io` path to the running-user directory &mdash; `v3io://users/<running user>`.

The data containers and their contents are referenced differently depending on the programming interface.
For example:

- In local file-system (FS) commands you use the predefined `v3io` root data mount &mdash; `/v3io/<container name>[/<data path>]`.
  There's also a predefined local-FS `User` mount to the **users/&lt;running user&gt;** directory, and you can use the aforementioned environment variables when setting data paths.
  For example, `/v3io/users/$V3IO_USERNAME`, `/v3io/$V3IO_HOME`, and `/User` are all valid ways of referencing the **users/&lt;running user&gt;** directory from a local FS command.
- In Hadoop FS or Spark DataFrame commands you use a fully qualified path of the format `v3io://<container name>/<data path>`.
  You can also use environment variables with these interfaces.

For detailed information and examples on how to set the data path for each interface, see [Setting Data Paths](https://www.iguazio.com/docs/latest-release/tutorials/getting-started/fundamentals/#data-paths) and the examples in the platform's tutorial Jupyter notebooks.

<a id="gs-data-collection-and-ingestion"></a>
## Collecting and Ingesting Data

The platform supports various alternative methods for collecting and ingesting data into its data containers (i.e., its data store).
For more information, see the [**platform-overview.ipynb**](../platform-overview.ipynb.ipynb#data-collection-and-ingestion) tutorial notebook
The data collection and ingestion can be done as a one-time operation, using different platform APIs &mdash; which can be run from your preferred programming interface, such as an interactive web-based Jupyter or Zeppelin notebook &mdash; or as an ongoing ingestion stream, using Nuclio serverless functions.
This section explains and demonstrates how to collect and ingest (import) data into the platform using code that's run from a Jupyter notebook.

<a id="ingest-from-external-db-to-no-sql-using-frames"></a>
### Ingesting Data From an External Database to a NoSQL Table Using V3IO Frames

For an example of how to collect data from an external database &mdash; such as MySQL, Oracle, and PostgreSQL &mdash; and ingest (write) it into a NoSQL table in the platform, using the V3IO Frames API, see the [read-external-db](read-external-db.ipynb) tutorial.

<a id="ingest-from-amazon-s3"></a>
### Ingesting Files from Amazon S3

<a id="ingest-from-amazon-s3-using-curl"></a>
#### Ingesting Files from Amazon S3 to the Platform File System Using curl

You can use a simple [curl](https://curl.haxx.se/) command to ingest a file (object) from an external web data source, such as an Amazon S3 bucket, to the platform's distributed file system (i.e., into the platform's data store).
This is demonstrated in the following code example and in the [getting-started example](#getting-started-example) in this notebook.
The [spark-sql-analytics](spark-sql-analytics.ipynb) tutorial notebook demonstrates a similar ingestion using [Botocore](https://github.com/boto/botocore).

The example in the following cells uses curl to read a CSV file from the [Iguazio sample data-sets](http://iguazio-sample-data.s3.amazonaws.com/) public Amazon S3 bucket and save it to an **examples** directory in the running-user directory of the predefined "users" data container (`/v3io/users/$V3IO_USERNAME` = `v3io/$V3IO_HOME` = `/User`).

In [1]:
!mkdir -p /User/examples # <=> /v3io/${V3IO_HOME}/examples or /v3io/users/${V3IO_USERNAME}/examples

In [2]:
%%sh
CSV_PATH="/User/examples/stocks.csv"
curl -L "iguazio-sample-data.s3.amazonaws.com/2018-03-26_BINS_XETR08.csv" > ${CSV_PATH}

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  861k  100  861k    0     0   493k      0  0:00:01  0:00:01 --:--:--  493k


<a id="ingest-from-amazon-s3-to-nosql-table-using-v3io-frames-n-pandas"></a>
#### Ingesting Data from Amazon S3 to a NoSQL Table Using V3IO Frames and pandas

For an example of how to import data from Amazon S3 and save it into a NoSQL table in the platform's data store by using V3IO Frames and pandas DataFrames, see the [frames](frames.ipynb) tutorial notebook.

<a id="streaming-data-from-an-external-streaming-engine-using-nuclio"></a>
### Streaming Data From an External Streaming Engine Using Nuclio

To read data from an external streaming engine &mdash; such as Kafka, Kinesis, or RabbitMQ &mdash; create a Nuclio function that listens on the stream, and write the stream data to a NoSQL or time-series database (TSDB) table:

1. In the dashboard's side navigation menu, select **Functions** to display the Nuclio serverless functions dashboard.
2. Create a new Nuclio project or select an existing project.
3. In the action toolbar, select **Create Function**.
4. Enter a function name, select an appropriate template, such as **kafka-to-tsdb**, configure the required template parameters, and apply your changes.
5. Select **Deploy** from the action toolbar to deploy your function.

<a id="gs-data-exploration-and-processing"></a>
## Exploring and Processing Data

After you have ingested data into the platform's data containers, you can use various alternative methods and tools to explore and analyze the data.
Data scientists typically use Jupyter Notebook to run the exploration phase.
As outlined in the [**welcome**](../welcome.ipynb#data-exploration-and-processing) tutorial notebook, the platform's Jupyter Notebook service has a wide range of pre-deployed popular data science tools (such as Spark and Presto) and allows installation of additional tools and packages, enabling you to use different APIs to access the same data from a single Jupyter notebook.
This section explains and demonstrates how to explore data in the platform from a Jupyter notebook.

<a id="data-exploration-spark"></a>
### Exploring Data using Spark DataFrames

Spark is a distributed computing framework for data analytics.
You can easily run distributed Spark jobs on you platform cluster that use Spark DataFrames to access data files (objects), tables, or streams in the platform's data store.
For more information and examples, see the [spark-sql-analytics](spark-sql-analytics.ipynb) tutorial notebook.

<a id="data-exploration-v3io-frames-n-pandas"></a>
### Exploring Data Using V3IO Frames and pandas DataFrames

Iguazio's V3IO Frames open-source data-access library provides a unified high-performance DataFrames API for accessing NoSQL, stream, and time-series data in the platform's data store.
These DataFrames can also be used to analyze the data with pandas. 
For details and examples, see the [frames](frames.ipynb) tutorial notebook.

<a id="data-exploration-sql"></a>
### Exploring Data Using SQL

You can run SQL statements (`SELECT` only) on top of NoSQL tables in the platform's data store.
To do this, you need to use the Jupyter `%sql` or `%%sql` IPython Jupyter magic followed by an SQL statement.
The platform supports standard ANSI SQL semantics.
Under the hood, the SQL statements are executed via [Presto](https://prestosql.io/), which is a distributed SQL engine designed from the ground up for fast analytics queries.

In the example in the following cell, as a preparation for the SQL query, the **stocks.csv** file that was ingested to the **users/&lt;running user&gt;/examples** platform data-container directory in the previous [Ingesting Files from Amazon S3 to the Platform](#ingest-from-amazon-s3) example is written to a **stocks_example_tab** NoSQL table in the same directory.
Then, an SQL `SELECT` query is run on this table.
You can also find a similar example in the [getting-started example](#getting-started-example) in this notebook.

In [3]:
# Use V3IO Frames to convert the CSV file that was ingested in the AWS S3 data-collection example to a NoSQL table.
# NOTE: Make sure to first create a V3IO Frames service from the "Services" page of the platform dashboard, and run the
# "Ingesting Files from Amazon S3 to the Platform File System Using curl" example to create users/$V3IO_USERNAME/examples/stocks.csv.
import pandas as pd
import v3io_frames as v3f
import os

# Create a V3IO Frames client for the "users" data container
client = v3f.Client("framesd:8081", container="users")

# Full CSV file path
csv_path = os.path.join("/User", "examples", "stocks.csv")
# Relative NoSQL table path within the "users" container
rel_nosql_table_path = os.path.join(os.getenv('V3IO_USERNAME'), "examples", "stocks_example_tab")

# Read the CSV file into a Pandas DataFrame
df = pd.read_csv(csv_path, header="infer")

# Convert the CSV file to a NoSQL table
client.write("kv", rel_nosql_table_path, df)

In [4]:
# Use Presto to query the NoSQL table that was created in the previous step
presto_nosql_table_path = os.path.join('v3io.users."' + os.getenv('V3IO_USERNAME'), 'examples', 'stocks_example_tab"')
%sql select * from $presto_nosql_table_path limit 10

Done.


securitydesc,securitytype,time,isin,minprice,date,endprice,numberoftrades,mnemonic,currency,securityid,idx,maxprice,tradedvolume,startprice
XTR.DAX INCOME 1D,ETF,08:58,LU0838782315,105.16,2018-03-26,105.16,2,XDDX,EUR,2504277,7175,105.16,172,105.16
"AMAZON.COM INC. DL-,01",Common stock,08:54,US0231351067,1235.99,2018-03-26,1235.99,2,AMZ,EUR,2506427,6714,1235.99,5,1235.99
AURUBIS AG,Common stock,08:54,DE0006766504,66.82,2018-03-26,66.82,2,NDA,EUR,2505060,6673,66.82,127,66.82
WACKER CHEMIE O.N.,Common stock,08:48,DE000WCH8881,136.45,2018-03-26,136.85,13,WCH,EUR,2504860,5893,136.85,1015,136.45
THYSSENKRUPP AG O.N.,Common stock,08:41,DE0007500001,21.45,2018-03-26,21.47,17,TKA,EUR,2505105,5180,21.47,9298,21.45
UNIPER SE NA O.N.,Common stock,08:35,DE000UNSE018,24.17,2018-03-26,24.25,15,UN01,EUR,2504856,4415,24.25,3361,24.17
BRENNTAG AG NA O.N.,Common stock,08:33,DE000A1DAHH0,48.18,2018-03-26,48.18,2,BNR,EUR,2504453,4130,48.18,183,48.18
MUTARES AG,Common stock,08:30,DE000A0SMSH2,18.25,2018-03-26,18.35,2,MUX,EUR,2504399,3777,18.35,954,18.25
SYMRISE AG INH. O.N.,Common stock,08:28,DE000SYM9999,64.32,2018-03-26,64.32,3,SY1,EUR,2504852,3536,64.34,511,64.34
VOLKSWAGEN AG ST O.N.,Common stock,08:23,DE0007664005,157.2,2018-03-26,157.3,5,VOW,EUR,2505113,3016,157.3,377,157.3


<a id="getting-started-example"></a>
## Data Collection and Exploration Getting-Started Example

This section demonstrates a data collection, ingestion, and exploration flow.
Follow the tutorial by running the code cells in order of appearance:
- [Step #1](#getting-started-example-step-ingest-csv) &mdash; a CSV file is read from an Amazon S3 bucket and saved into an examples data-container directory using curl.
  The examples directory is first created by using a file-system command.
- [Step #2](#getting-started-example-step-convert-csv-to-nosql-table) &mdash; the ingested file is converted into a NoSQL table by using Spark DataFrames.
- [Step #3](#getting-started-example-step-run-sql-queries) &mdash; a Presto SQL query is run on the NoSQL table.
- [Step #4](#getting-started-example-step-convert-data-to-parquet) &mdash; the ingested CSV file is converted into a Parquet table by using Spark DataFrames.
- [Step #5](#getting-started-example-step-browse-the-examples-dir) &mdash; the examples container directory is browsed by using local and Hadoop file-system commands.
- At the end of the flow, you can optionally [delete](#getting-started-example-deleting-data) the examples directory using a file-system command.

You can find more information about this sample flow in the [Converting a CSV File to a NoSQL Table](https://www.iguazio.com/docs/latest-release/tutorials/getting-started/ingest-n-consume-files/#convert-csv-to-nosql) platform quick-start tutorial.

> **Tip:** You can also browse the files and directories that you write to the "users" container in this tutorial from the platform dashboard: in the side navigation menu, select **Data**, and then select the **users** container from the table. On the container data page, select the **Browse** tab, and then use the side directory-navigation tree to browse the directories. Selecting a file or directory in the browse table displays its metadata.

<a id="getting-started-example-step-ingest-csv"></a>
### Step 1: Ingest a Sample CSV File from Amazon S3

Use `curl` to download a sample stocks-data CSV file from the [Iguazio sample data-set](http://iguazio-sample-data.s3.amazonaws.com/) public Amazon S3 bucket.
For additional public data sets, check out [Registry of Open Data on AWS](https://registry.opendata.aws/).

> **NOTE:** All the platform tutorial notebook examples store the data in an **examples** directory in the running-user directory of the predefined "users" container &mdash; **users/&lt;running user&gt;/examples**.
> The running-user directory is automatically created by the Jupyter Notebook service.
> The `V3IO_HOME` environment variable is used to reference the **users/&lt;running user&gt;** directory.
> To save the data to a different root container directory or to a different container, you need to specify the data path in the local file-system commands as `/v3io/<container name>/<data path>`, and in Spark DataFrames or Hadoop FS commands as a fully qualified path of the format `v3io://<container name>/<table path>`.
> For more information, see the [v3io-mount](#v3io-mount) and [running-user directory](#running-user-dir) information [Jupyter Notebook Basics](#jupyter-notebook-basics) section of this notebook.

In [5]:
%%sh
DIR_PATH="/User/examples/" # <=> "/v3io/${V3IO_HOME}/examples/" or "/v3io/users/${V3IO_USERNAME}/examples/"
CSV_PATH="${DIR_PATH}stocks.csv"

# Create the examples directory
mkdir -p ${DIR_PATH}

# Download a sample stocks CSV file from the Iguazio sample data-set Amazon S3 bucket to the examples directory
curl -L "iguazio-sample-data.s3.amazonaws.com/2018-03-26_BINS_XETR08.csv" > ${CSV_PATH}

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  861k  100  861k    0     0   592k      0  0:00:01  0:00:01 --:--:--  592k


<a id="getting-started-example-step-convert-csv-to-nosql-table"></a>
### Step 2: Convert the Sample CSV File to a NoSQL Table

Read the sample **stocks.csv** file that you downloaded and ingested in the previous step into a Spark DataFrame, and write the data in NoSQL format to a new "stocks_tab" table in the same container directory (**users/&lt;running user&gt;/examples/stocks_tab**). 

> **Note**
> - To use the Iguazio Spark Connector, set the DataFrame's data-source format to `io.iguaz.v3io.spark.sql.kv`.
> - The data path in the Spark DataFrame is specified by using the `V3IO_HOME_URL` environment variable, which is set to `v3io://users/<running user>`.
>   See the [running-user directory](#running-user-dir) information.

In [6]:
import os
from pyspark.sql import SparkSession

# Example diretory path - a <running user>/examples directory in the "users" container
dir_path = os.path.join(os.getenv("V3IO_HOME_URL"), "examples")
# CSV file path
csv_path = os.path.join(dir_path, "stocks.csv")
# NoSQL table path
nosql_table_path = os.path.join(dir_path, "stocks_tab")

# Create a new Spark session
spark = SparkSession.builder.appName("Iguazio data ingestion and preparation getting-started example").getOrCreate()

# Read the sample CSV file into a Spark DataFrame, and let Spark infer the schema of the data
df = spark.read.option("header", "true").csv(csv_path, inferSchema="true")

# Show the DataFrame data
df.show()

# Write the DataFrame data to a NoSQL table in a platform data container.
# Define the "ISIN" column (attribute) as the table's primary key.
df.write.format("io.iguaz.v3io.spark.sql.kv").mode("append") \
    .option("key", "ISIN").option("allow-overwrite-schema", "true") \
    .save(nosql_table_path)

# Display the table schema:
df.printSchema()

+------------+--------+--------------------+------------+--------+----------+-------------------+-----+----------+--------+--------+--------+------------+--------------+
|        ISIN|Mnemonic|        SecurityDesc|SecurityType|Currency|SecurityID|               Date| Time|StartPrice|MaxPrice|MinPrice|EndPrice|TradedVolume|NumberOfTrades|
+------------+--------+--------------------+------------+--------+----------+-------------------+-----+----------+--------+--------+--------+------------+--------------+
|CH0038389992|    BBZA|BB BIOTECH NAM.  ...|Common stock|     EUR|   2504244|2018-03-26 00:00:00|08:00|      56.4|    56.4|    56.4|    56.4|         320|             4|
|CH0038863350|    NESR|NESTLE NAM.      ...|Common stock|     EUR|   2504245|2018-03-26 00:00:00|08:00|     63.04|   63.06|    63.0|   63.06|         314|             3|
|LU0378438732|    C001|COMSTAGE-DAX UCIT...|         ETF|     EUR|   2504271|2018-03-26 00:00:00|08:00|    113.42|  113.42|  113.42|  113.42|         

<a id="getting-started-example-step-run-sql-queries"></a>
### Step 3: Run Interactive SQL Queries

Use the `%sql` Jupyter magic to run an SQL queries on the "stocks_tab" table that was created in the previous step.
(The queries is processed using Presto.)
The example runs a `SELECT` query that reads the first ten table items.

In [7]:
presto_nosql_table_path = os.path.join('v3io.users."' + os.getenv('V3IO_USERNAME'), 'examples', 'stocks_tab"')

In [8]:
%sql select * from $presto_nosql_table_path limit 10

Done.


securitydesc,securitytype,time,isin,minprice,date,endprice,numberoftrades,mnemonic,currency,securityid,maxprice,tradedvolume,startprice
LINDE AG O.N.,Common stock,08:04,DE0006483001,168.4,2018-03-26 00:00:00.000,168.4,1,LIN,EUR,2505043,168.4,74,168.4
IMMOFINANZ AG INH.,Common stock,08:54,AT0000809058,2.084,2018-03-26 00:00:00.000,2.084,1,IMO,EUR,2504179,2.084,1000,2.084
CLIQ DIGITAL AG O.N.,Common stock,08:08,DE000A0HHJR3,5.74,2018-03-26 00:00:00.000,5.74,1,CLIQ,EUR,2504310,5.74,196,5.74
ISHARES SLI UCITS ETF DE,ETF,08:56,DE0005933964,80.6,2018-03-26 00:00:00.000,80.6,1,EXI1,EUR,2504990,80.6,75,80.6
ISHSV.MSCI W.EUR HGD ACC,ETF,08:00,IE00B441G979,48.902,2018-03-26 00:00:00.000,48.902,1,IBCH,EUR,2505679,48.902,1368,48.902
"STEINHOFF INT.HLDG.EO-,50",Common stock,08:00,NL0011375019,0.236,2018-03-26 00:00:00.000,0.2393,42,SNH,EUR,2506267,0.2405,220338,0.2378
SOURCE-S.ST.E.600OP.PHG A,ETF,08:06,IE00B5MTZ595,449.55,2018-03-26 00:00:00.000,449.55,1,SC04,EUR,2505707,449.55,38,449.55
DEMIRE DT.MTS.RE AG,Common stock,08:19,DE000A0XFSF0,4.42,2018-03-26 00:00:00.000,4.42,2,DMRE,EUR,2504430,4.42,25075,4.42
EPIGENOMICS AG NA O.N.,Common stock,08:07,DE000A11QW50,3.78,2018-03-26 00:00:00.000,3.78,1,ECX,EUR,2504547,3.78,100,3.78
LYX.EURO ST.50D.(2X)L. A,ETF,08:08,FR0010468983,23.02,2018-03-26 00:00:00.000,23.02,1,LYMZ,EUR,2506387,23.02,2800,23.02


In [9]:
%sql select count(*) from $presto_nosql_table_path 

Done.


_col0
737


<a id="getting-started-example-step-convert-data-to-parquet"></a>
### Step 4: Convert the Data to a Parquet Table

Use a Spark DataFrame `write` command to write the data in the Spark DaraFrame &mdash; which was created from the CSV file and used to create the NoSQL table in [Step 2](#getting-started-example-step-convert-csv-to-nosql-table) &mdash; to a new **users/&lt;running user&gt;/examples/stocks_prqt** Parquet table.

In [10]:
# Write the DataFrame data that was read from the CSV file in Step 2 to a Parquet table in a platform data container
prqt_table_path = os.path.join(dir_path, "stocks_prqt")
df.write.mode('overwrite').parquet(prqt_table_path)

<a id="getting-started-example-step-browse-the-examples-dir"></a>
### Step 5: Browse the Example Container Directory

Use a file-system bash-shell command to list the contents of the **users/&lt;running user&gt;/examples** data-container directory to which all the ingested data in the previous steps were saved.
You should see in this directory the **stocks.csv** file, **stocks_tab** NoSQL table directory, and **stocks_prqt** Parquet table directory that you created in the previous steps.
The following cells demonstrate how to issue the same command using the local file system and using Hadoop FS.

In [11]:
# List the contents of the users/<running user>/examples directory using a local file-system command
!ls -lrt /User/examples
# The following are equivalent commands that demonstrate different ways to reference your user home directory:
#!ls -lrt /v3io/${V3IO_HOME}/examples
#!ls -lrt /v3io/users/${V3IO_USERNAME}/examples

total 0
drwxrwxr-x 2 50 nogroup      0 Mar 10 07:43 stocks_example_tab
-rw-r--r-- 1 50 nogroup 882055 Mar 10 07:45 stocks.csv
drwxrwxrwx 2 50 nogroup      0 Mar 10 07:47 stocks_tab
drwxr-xr-x 2 50 nogroup      0 Mar 10 07:49 stocks_prqt
-rw-r--r-- 1 50 nogroup 150996 Mar 10 08:38 parquet_example
-rw-rw-r-- 1 50 nogroup 113629 Mar 10 08:48 userdata1.parquet
drwxrwxr-x 2 50 nogroup      0 Mar 10 08:50 spark-output
drwxrwxr-x 2 50 nogroup      0 Mar 10 08:51 multiple-parquet-files
drwxrwxr-x 2 50 nogroup      0 Mar 10 09:18 family
drwxrwxr-x 2 50 nogroup      0 Mar 10 09:19 family1
drwxrwxr-x 2 50 nogroup      0 Mar 10 09:19 family2
-rw-rw-r-- 1 50 nogroup    475 Mar 10 09:43 mLines.json
-rw-rw-r-- 1 50 nogroup 131745 Mar 10 09:46 CoffeeTime.jpg
drwxrwxrwx 2 50 nogroup      0 Mar 10 09:47 mytable
drwxrwxrwx 2 50 nogroup      0 Mar 10 09:48 stocks_kv
drwxrwxrwx 2 50 nogroup      0 Mar 10 09:48 stocks_kv_partition
drwxr-xr-x 2 50 nogroup      0 Mar 10 09:52 stocks_parq
drwxrwxrwx 2 50 nogro

In [12]:
%%sh
# List the contents of the users/<running user>/examples directory using an Hadoop FS command
hadoop fs -ls ${V3IO_HOME_URL}/examples
# The following are equivalent commands that demonstrate different ways to reference your user home directory:
#hadoop fs -ls v3io://${V3IO_HOME}/examples
#hadoop fs -ls v3io://users/${V3IO_USERNAME}/examples

Found 4 items
-rw-r--r--   1 55 root     882055 2020-01-16 09:00 v3io://users/iguazio/examples/stocks.csv
drwxrwxr-x   - 55 root          0 2020-01-16 09:00 v3io://users/iguazio/examples/stocks_example_tab
drwxr-xr-x   - 55 root          0 2020-01-16 09:01 v3io://users/iguazio/examples/stocks_prqt
drwxrwxrwx   - 55 root          0 2020-01-16 09:01 v3io://users/iguazio/examples/stocks_tab


20/01/16 09:01:24 INFO slf_4j.Slf4jLogger: Slf4jLogger started


<a id="cleanup"></a>
## Cleanup

Prior to exiting, release disk space, computation, and memory resources consumed by the active session:

- [Delete Data](#delete-data)
- [Release Spark Resources](#release-spark-resources)

<a id="delete-data"></a>
### Delete Data

Optionally delete  any of the directories or files that you created.
See the instructions in the [Creating and Deleting Container Directories](https://www.iguazio.com/docs/latest-release/tutorials/getting-started/containers/#create-delete-container-dirs) tutorial.
The following example uses a local file-system command to delete the entire contents of the **users/&lt;running user&gt;/examples** directory that was created in this example, but not the directory itself.

In [13]:
# Delete the contents of the examples directory:
#!rm -rf /User/examples/*
# You can also delete the examples directory iteself (and all its contents):
#!rm -rf /User/examples/

<a id="release-spark-resources"></a>
### Release Spark Resources

When you're done, run the following command to stop your Spark session and release its computation and memory resources:

In [14]:
spark.stop()